# LDA topic modelling
The notebook on LDA topic modelling for IMDB dataset using PySpark is a comprehensive guide that demonstrates how to perform topic modelling using Latent Dirichlet Allocation (LDA) in PySpark.

In [7]:
import findspark
findspark.init()
from pyspark.sql import functions as F
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext("local[*]")
spark = SparkSession.builder.getOrCreate()

In [94]:
# There are some warrings, so we supress them
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load and preprocess the data

In [129]:
import re
import numpy as np
tFile="data\IMDB Dataset.csv.bz2"
df = spark.read.csv(tFile,header=True)
df.show(3)

+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|One of the other ...| positive|
|A wonderful littl...| positive|
|I thought this wa...| positive|
+--------------------+---------+
only showing top 3 rows



In [130]:
df = df.sample(.2, seed=100)
#df= df.where(F.col("sentiment")=="positive")

In [131]:
df.groupBy("sentiment").count().show()

+---------+-----+
|sentiment|count|
+---------+-----+
| positive| 5067|
| negative| 4993|
+---------+-----+



In [132]:
# Remove html tags from text
df = df.withColumn("text_c", F.regexp_replace(F.col("text"), r'<[^>]+>', ""));
# Remove non-letters
df = df.withColumn("text_c", F.regexp_replace("text_c", r"[\.\!\,\-\']", " "))
# Remove non-letters
df = df.withColumn("text_c", F.regexp_replace("text_c", r"[^a-zA-Z\ ]", ""))
# Remove words 1, 2 char
df = df.withColumn("text_c", F.regexp_replace("text_c", r"\b\w{1,2}\b", " "))
df.toPandas().head(5)

,text,sentiment,text_c
0,A wonderful little production. <br /><br />The...,positive,wonderful little production The filming tec...
1,"Probably my all-time favorite movie, a story o...",positive,Probably all time favorite movie story ...
2,I sure would like to see a resurrection of a u...,positive,sure would like see resurrection d...
3,"This show was an amazing, fresh & innovative i...",negative,This show was amazing fresh innovative ide...
4,The cast played Shakespeare.<br /><br />Shakes...,negative,The cast played Shakespeare Shakespeare lost ...


### Lemmatization (optional)
Lemmatization is the process of reducing a word to its base or root form, which is also known as a lemma. The purpose of lemmatization is to simplify text and make it easier to analyze by grouping together different forms of the same word. For example, the words "running," "ran," and "runs" can all be reduced to the base form "run" through lemmatization. 

However, lemmatization can be a **time-consuming operation**, especially when dealing with large amounts of text data. This is because the process involves analyzing each word in a text and identifying its base form. It also requires a comprehensive understanding of the grammatical rules of a language to accurately identify the correct lemma for each word.

Despite its time-consuming nature, lemmatization can be a powerful tool in natural language processing and text analysis. It can help with tasks such as sentiment analysis, topic modeling, and text classification. When using lemmatization, it's important to use it carefully and correctly to ensure that the text is properly processed and analyzed.

In [133]:
import spacy
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Load the spaCy model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Define a function to apply the lemmatizer to a text
def lemmatize_text(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

# Define a UDF to apply the lemmatizer to a column
lemmatize_udf = udf(lemmatize_text, StringType())

# Apply the UDF to a DataFrame column
df = df.withColumn("text_c", lemmatize_udf(df["text_c"]))

# Caching must be used !!!!!!
df = df.cache()
df.toPandas().head(5)

,text,sentiment,text_c
0,A wonderful little production. <br /><br />The...,positive,wonderful little production the film tech...
1,"Probably my all-time favorite movie, a story o...",positive,probably all time favorite movie story ...
2,I sure would like to see a resurrection of a u...,positive,sure would like see resurrection ...
3,"This show was an amazing, fresh & innovative i...",negative,this show be amazing fresh innovative i...
4,The cast played Shakespeare.<br /><br />Shakes...,negative,the cast play Shakespeare Shakespeare lose ...


In [64]:
from pyspark.ml.feature import Tokenizer, CountVectorizer,IDF
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.clustering import LDA

# Preprocessing Pipeline
Preprocessing pipeline for text classification using LDA may include the following steps:

- Tokenization: This step involves breaking the text into individual words or tokens. A tokenizer can split the text into tokens based on spaces, punctuation, and other delimiters.
- Stop words removal: This step involves removing commonly used words in a language that do not carry much meaning or contribute to the topic of the text, such as "the", "and", "is". Stop words removal helps to reduce the size of the vocabulary and improves the efficiency of subsequent steps.
- Count vectorizer: This step involves converting the tokenized text into a matrix of word counts. Each row represents a document, and each column represents a unique word in the vocabulary. The values in the matrix represent the frequency of each word in each document.
- IDF (Inverse Document Frequency): This step involves weighting the word counts to account for the frequency of each word in the entire corpus. Words that occur frequently across all documents are given a lower weight, while words that occur rarely are given a higher weight.

By using IDF, the weight of each word is inversely proportional to the number of documents that contain that word. This helps to identify the important words that are specific to a particular document and can help to distinguish between different topics.

Once these steps are completed, the resulting TF-IDF vectors can be used as input for the LDA algorithm to identify the underlying topics in the text corpus and classify the documents based on their topic distributions.

In [163]:
# Text preprocessin pipeline
tokenizer = Tokenizer(inputCol="text_c", outputCol="words")
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
# Run 1 Use 500 words
countVectorizer = CountVectorizer(inputCol=remover.getOutputCol(), outputCol="features_c", vocabSize=500)
# Run 2 Use 1000 words
#countVectorizer = CountVectorizer(inputCol=remover.getOutputCol(), outputCol="features_c", vocabSize=1000)
# Run 3 Use Filter most frequent words
#countVectorizer = CountVectorizer(inputCol=remover.getOutputCol(), outputCol="features_c", vocabSize=1000,minDF=10, maxDF=5000)

idf = IDF(inputCol=countVectorizer.getOutputCol(), outputCol="features")
pipeline = Pipeline(stages=[tokenizer,remover, countVectorizer,idf])
data_model = pipeline.fit(df)

In [164]:
# Print the vaocabulary
vocabulary = data_model.stages[2].vocabulary
print(vocabulary[:100])

['', 'movie', 'film', 'one', 'see', 'make', 'like', 'good', 'get', 'well', 'time', 'character', 'watch', 'bad', 'even', 'story', 'really', 'think', 'show', 'scene', 'great', 'look', 'much', 'say', 'know', 'people', 'go', 'also', 'take', 'give', 'first', 'way', 'end', 'love', 'thing', 'play', 'come', 'find', 'man', 'life', 'seem', 'work', 'actor', 'plot', 'two', 'year', 'many', 'want', 'never', 'little', 'try', 'ever', 'act', 'still', 'feel', 'back', 'part', 'use', 'something', 'old', 'real', 'funny', 'lot', 'director', 'didn', 'guy', 'woman', 'performance', 'leave', 'star', 'big', 'another', 'doesn', 'role', 'though', 'young', 'nothing', 'actually', 'start', 'tell', 'point', 'new', 'long', 'day', 'every', 'cast', 'world', 'become', 'girl', 'fact', 'turn', 'comedy', 'pretty', 'horror', 'set', 'action', 'kill', 'however', 'enough', 'minute']


In [165]:
# Transform the dataset using the preprocessing pipeline
dataset = data_model.transform(df)
dataset.toPandas().tail(5)

,text,sentiment,text_c,words,filtered,features_c,features
10055,"As someone who loves baseball history, especia...",negative,someone who love baseball history especia...,"[, , , someone, who, love, baseball, history, ...","[, , , someone, love, baseball, history, , , e...","(237.0, 1.0, 2.0, 1.0, 2.0, 3.0, 1.0, 0.0, 0.0...","(0.023557477281972106, 0.4314942162337284, 1.0..."
10056,<br /><br />Headlines warn us of the current c...,positive,headline warn the current campaign dem...,"[headline, warn, , , , , , the, current, campa...","[headline, warn, , , , , , current, campaign, ...","(167.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0...","(0.016599572599533086, 0.0, 0.0, 0.0, 1.244164..."
10057,"Dog Bite Dog isn't going to be for everyone, b...",positive,Dog Bite Dog isn go for everyone but...,"[dog, bite, dog, isn, , , , go, , , , , , for,...","[dog, bite, dog, isn, , , , go, , , , , , ever...","(285.0, 0.0, 2.0, 2.0, 1.0, 0.0, 2.0, 1.0, 0.0...","(0.02832861192135886, 0.0, 1.0698027149082745,..."
10058,This is your typical junk comedy.<br /><br />T...,negative,this your typical junk comedy there be almo...,"[this, , , , your, typical, junk, comedy, ther...","[, , , typical, junk, comedy, almost, , , , la...","(132.0, 4.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0...","(0.013120620258313578, 1.7259768649349136, 0.5..."
10059,I thought this movie did a down right good job...,positive,think this movie do down right good job ...,"[, , , think, this, movie, do, , , , down, rig...","[, , , think, movie, , , , right, good, job, ,...","(204.0, 6.0, 0.0, 1.0, 3.0, 0.0, 2.0, 1.0, 2.0...","(0.02027732221739371, 2.5889652974023702, 0.0,..."


# LDA
LDA (Latent Dirichlet Allocation) is a topic modeling technique that is used to identify the underlying topics within a large collection of text documents. It is a probabilistic model that assumes each document is a mixture of several topics and each topic is a mixture of several words.
- The LDA algorithm works by first randomly assigning each word in each document to a topic. It then iteratively improves these assignments by reassigning words to topics based on the probability of the word belonging to each topic, and the probability of each document belonging to each topic.
- The output of the LDA algorithm is a set of topics, each represented by a distribution over words, and a set of document-topic distributions, which represent the degree to which each document belongs to each topic.
- LDA can be used for a wide range of applications, such as content analysis, information retrieval, and recommendation systems. It has been widely applied in fields such as social media analysis, e-commerce, and market research.
- One of the main advantages of LDA is that it can automatically identify the topics present in a corpus without any prior knowledge of the topics. This makes it a useful tool for analyzing large and complex datasets.
- However, LDA has some limitations. It may not work well with short documents, and it may require a large amount of data to accurately estimate the topic distributions. Additionally, the interpretation of the resulting topics may require human expertise and domain knowledge.

In [166]:
# Find two topics
lda = LDA(k=2, maxIter=20)
model = lda.fit(dataset)

### Topics Matrix
Inferred topics, where each topic is represented by a distribution over terms. This is a matrix of size vocabSize x k, where each column is a topic. No guarantees are given about the ordering of the topics.

Warning: If this model is actually a DistributedLDAModel instance produced by the Expectation-Maximization (“em”) optimizer, then this method could involve collecting a large amount of data to the driver (on the order of vocabSize x k).

In [170]:
# Print the LDA transformation matrix
print(model.topicsMatrix().toArray().shape)
model.topicsMatrix().toArray()

(500, 2)


array([[  43.55667383,   42.23974876],
       [1700.17670733, 2287.48165437],
       [2307.39672356, 2273.47952778],
       [1325.65910178, 1446.28216157],
       [1065.76323985, 1601.95282078],
       [1259.65631661, 1467.14081886],
       [1069.1256844 , 1673.7189798 ],
       [1206.71126969, 1565.12820087],
       [1319.15256515, 1373.65887794],
       [1193.97577328, 1329.38320978],
       [1053.25460409, 1482.43667296],
       [1400.89893967, 1378.55972731],
       [ 701.77612693, 1762.76753745],
       [1138.7736189 , 1687.03642088],
       [1019.5383721 , 1445.55022416],
       [1478.74191773, 1254.36484414],
       [ 929.19902861, 1583.37523568],
       [ 983.18231853, 1405.0901764 ],
       [ 903.29766687, 2008.26064517],
       [1142.94973704, 1313.24440841],
       [1036.49280297, 1304.76451474],
       [1160.52334713, 1170.78776604],
       [ 960.933437  , 1222.95421167],
       [ 920.15427162, 1342.09490669],
       [1083.58523715, 1137.71512003],
       [1087.51312237, 12

### Describe topics
Return the topics described by weighted terms. Each topic is represented as a pair of matching arrays: (term indices, term weights in topic). Each topic’s terms are sorted in order of decreasing weight.

In [139]:

topics = model.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics.toPandas().head(5)

The topics described by their top-weighted terms:


,topic,termIndices,termWeights
0,0,"[2, 1, 15, 135, 38]","[0.008111784371918734, 0.006281720343851382, 0..."
1,1,"[2, 1, 18, 12, 4]","[0.00847788352173735, 0.008178312170032061, 0...."


In [142]:
# Print most important words per topic
topics = model.describeTopics(15)
for r in topics.select("termIndices").collect():
    rez = []
    for l in r:
        for i in l:
            rez.append(vocabulary[i])
    print(rez[:15])

['film', 'movie', 'story', 'book', 'man', 'woman', 'horror', 'get', 'character', 'bad', 'one', 'make', 'look', 'good', 'girl']
['film', 'movie', 'show', 'watch', 'see', 'like', 'good', 'love', 'funny', 'really', 'bad', 'time', 'one', 'make', 'character']


# Create LDA model wiht ten topics

In [149]:
# Text preprocessin pipeline
tokenizer = Tokenizer(inputCol="text_c", outputCol="words")
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
# Run 1: Use all the words
# countVectorizer = CountVectorizer(inputCol=remover.getOutputCol(), outputCol="features_c", vocabSize=1000)
# Run 2: Discard the very frequent words
countVectorizer = CountVectorizer(inputCol=remover.getOutputCol(), outputCol="features_c", vocabSize=1000,minDF=10, maxDF=3000)

idf = IDF(inputCol=countVectorizer.getOutputCol(), outputCol="features")
pipeline = Pipeline(stages=[tokenizer,remover, countVectorizer,idf])
data_model = pipeline.fit(df)

In [150]:
vocabulary = data_model.stages[2].vocabulary
print(vocabulary[:100])

['bad', 'show', 'scene', 'great', 'look', 'much', 'say', 'know', 'people', 'go', 'also', 'take', 'give', 'first', 'way', 'end', 'love', 'thing', 'play', 'come', 'find', 'man', 'life', 'seem', 'work', 'actor', 'plot', 'year', 'two', 'many', 'want', 'never', 'little', 'try', 'ever', 'act', 'still', 'feel', 'back', 'part', 'use', 'something', 'old', 'real', 'funny', 'lot', 'director', 'didn', 'guy', 'woman', 'performance', 'leave', 'star', 'another', 'big', 'doesn', 'role', 'though', 'young', 'nothing', 'actually', 'start', 'tell', 'point', 'new', 'long', 'day', 'every', 'cast', 'world', 'become', 'girl', 'fact', 'turn', 'comedy', 'pretty', 'horror', 'set', 'action', 'kill', 'however', 'enough', 'around', 'minute', 'quite', 'far', 'enjoy', 'may', 'happen', 'right', 'need', 'friend', 'without', 'bit', 'line', 'music', 'family', 'series', 'write', 'keep']


In [151]:
dataset = data_model.transform(df)
dataset.toPandas().head(5)

,text,sentiment,text_c,words,filtered,features_c,features
0,A wonderful little production. <br /><br />The...,positive,wonderful little production the film tech...,"[, , , wonderful, little, production, , , the,...","[, , , wonderful, little, production, , , film...","(0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.3378876461516889, 1.2959753371494..."
1,"Probably my all-time favorite movie, a story o...",positive,probably all time favorite movie story ...,"[probably, , , , all, time, favorite, movie, ,...","[probably, , , , time, favorite, movie, , , , ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2887367966762..."
2,I sure would like to see a resurrection of a u...,positive,sure would like see resurrection ...,"[, , , sure, would, like, , , , see, , , , res...","[, , , sure, like, , , , see, , , , resurrecti...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2887367966762..."
3,"This show was an amazing, fresh & innovative i...",negative,this show be amazing fresh innovative i...,"[this, show, be, , , , amazing, , , fresh, , ,...","[show, , , , amazing, , , fresh, , , innovativ...","(2.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.4281245920786856, 8.59418903791273, 0.0, 0...."
4,The cast played Shakespeare.<br /><br />Shakes...,negative,the cast play Shakespeare Shakespeare lose ...,"[the, cast, play, shakespeare, shakespeare, lo...","[cast, play, shakespeare, shakespeare, lose, ,...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2887367966762..."


In [171]:
# Find two topics
lda = LDA(k=10, maxIter=20)
model = lda.fit(dataset)

In [172]:
# Describe topics
topics = model.describeTopics(5)
print("The topics described by their top-weighted terms:")
topics.toPandas()

The topics described by their top-weighted terms:


,topic,termIndices,termWeights
0,0,"[335, 478, 424, 112, 127]","[0.011750275756590727, 0.011201784885636558, 0..."
1,1,"[254, 452, 139, 61, 183]","[0.010351669177697401, 0.00981643520814814, 0...."
2,2,"[33, 113, 161, 215, 296]","[0.014221778722554018, 0.01264231965702297, 0...."
3,3,"[93, 218, 226, 95, 300]","[0.011022784219911087, 0.010787888293754325, 0..."
4,4,"[290, 430, 115, 330, 351]","[0.022451135732642243, 0.015177460441792407, 0..."
5,5,"[188, 2, 39, 40, 38]","[0.015124622977892146, 0.010802576956638362, 0..."
6,6,"[2, 1, 277, 91, 66]","[0.011942007080201763, 0.009384165696750643, 0..."
7,7,"[18, 156, 65, 13, 88]","[0.017012833383330456, 0.016089205431532545, 0..."
8,8,"[413, 73, 67, 366, 216]","[0.01666269623142732, 0.015612766183368404, 0...."
9,9,"[135, 287, 1, 13, 42]","[0.022403329240157747, 0.014704018254128098, 0..."


In [173]:
# Print most important words per topic
topics = model.describeTopics(10)
for r in topics.select("termIndices").collect():
    rez = []
    for l in r:
        for i in l:
            rez.append(vocabulary[i])
    print(rez[:15])

['killer', 'musical', 'novel', 'music', 'original', 'song', 'film', 'version', 'dog', 'video']
['school', 'dance', 'kid', 'funny', 'black', 'movie', 'film', 'white', 'high', 'sad']
['love', 'family', 'child', 'father', 'mother', 'son', 'man', 'young', 'boy', 'daughter']
['horror', 'budget', 'low', 'action', 'flick', 'film', 'pretty', 'movie', 'gore', 'scene']
['game', 'sequel', 'series', 'buy', 'evil', 'fight', 'dvd', 'crap', 'michael', 'team']
['war', 'film', 'life', 'seem', 'man', 'sister', 'story', 'old', 'happen', 'world']
['film', 'movie', 'sex', 'comedy', 'woman', 'power', 'cinema', 'people', 'american', 'see']
['show', 'episode', 'guy', 'bad', 'girl', 'monster', 'get', 'like', 'stupid', 'watch']
['documentary', 'role', 'performance', 'support', 'excellent', 'film', 'drama', 'actress', 'actor', 'cast']
['book', 'terrible', 'movie', 'bad', 'actor', 'read', 'acting', 'good', 'character', 'look']


# Topic classification with LDA
LDA can be a useful technique for text classification, especially when you want to identify the underlying topics within a corpus and classify documents into those topics.

Advantages of using LDA for classification:
- Identifies latent topics: LDA can automatically identify the underlying topics in a corpus of text. This can be useful for discovering hidden themes and patterns in the data.
- Unsupervised learning: LDA is an unsupervised learning technique, which means that it does not require labeled data to identify the topics. This can save time and effort in preparing labeled data for training a classifier.
- Handles high-dimensional data: LDA can handle high-dimensional data, such as text documents, by reducing the dimensionality of the data to a smaller set of topics.
- Flexibility: LDA is a flexible technique that can be adapted to different types of text data and modeling assumptions.

Problems of using LDA for classification:
- Requires large data sets: LDA can require large data sets to accurately estimate the topic distributions. If the data set is too small, the resulting topic distributions may not be representative of the underlying data.
- Sensitivity to model parameters: LDA is sensitive to the number of topics and other model parameters. Choosing the optimal number of topics can be challenging and requires some trial and error.
- Limited interpretability: While LDA can identify latent topics, the resulting topics may not always be easily interpretable. It may require domain expertise to interpret the topics and assign meaningful labels to them.

In [158]:
# Shows the result
transformed = model.transform(dataset)
transformed.select("text_c","topicDistribution").toPandas().head(5)

,text_c,topicDistribution
0,wonderful little production the film tech...,"[0.09548062300683834, 0.2911398770813034, 0.00..."
1,probably all time favorite movie story ...,"[0.0026566711560415276, 0.7390297143612596, 0...."
2,sure would like see resurrection ...,"[0.4436860430498665, 0.24510181283072788, 0.00..."
3,this show be amazing fresh innovative i...,"[0.0013062027141876587, 0.31245572798942345, 0..."
4,the cast play Shakespeare Shakespeare lose ...,"[0.990110042621313, 0.002499987646618351, 0.00..."


In [159]:
from pyspark.sql.functions import udf
@udf
def vect_argmax(row):
    row_arr = row.toArray()
    max_pos = np.argmax(row_arr)
    return(int(max_pos))
transformed1 = transformed.withColumn("argmax",vect_argmax(F.col('topicDistribution')))

In [160]:
transformed1.select("text_c","argmax").toPandas().head(5)

,text_c,argmax
0,wonderful little production the film tech...,4
1,probably all time favorite movie story ...,1
2,sure would like see resurrection ...,0
3,this show be amazing fresh innovative i...,3
4,the cast play Shakespeare Shakespeare lose ...,0
